# 태양광 발전량 및 기상 실측 데이터 전처리

In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
# 데이터 불러오기
path = "./data/"

gens = pd.read_csv(path + "gens.csv")
j_gens = pd.read_csv(path+ "cmpt2021_data_july3/gens.csv")
weathers = pd.read_csv(path + "weathers1.csv")
j_weathers = pd.read_csv(path + "cmpt2021_data_july3/weathers1.csv")

gens['time'] = pd.to_datetime(gens['time']).dt.tz_convert('Asia/Seoul')
j_gens['time'] = pd.to_datetime(j_gens['time']).dt.tz_convert('Asia/Seoul')
weathers['time'] = pd.to_datetime(weathers['time']).dt.tz_convert('Asia/Seoul')
j_weathers['time'] = pd.to_datetime(j_weathers['time']).dt.tz_convert('Asia/Seoul')

gen_df = pd.concat([gens, j_gens])
wth_df = pd.concat([weathers, j_weathers])

df = pd.merge(gen_df, wth_df, how='left')
amount = df['amount']

In [3]:
df

,id,time,amount,temperature,humidity,dew_point,wind_dir,wind_spd,uv_idx,visibility,cloudiness,ceiling,pressure,precip_1h
0,1,2020-06-01 01:00:00+09:00,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-06-01 02:00:00+09:00,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-06-01 03:00:00+09:00,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-06-01 04:00:00+09:00,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2020-06-01 05:00:00+09:00,0.00000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,1010.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235004,24,2021-07-23 20:00:00+09:00,7.81948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235005,24,2021-07-23 21:00:00+09:00,0.06712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235006,24,2021-07-23 22:00:00+09:00,0.10068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235007,24,2021-07-23 23:00:00+09:00,0.13424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. 이상치 제거

In [4]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,235009.0,12.218187,6.765413,1.0,6.0,12.00000,18.000000,24.00000
amount,235009.0,126.640313,194.158092,0.0,0.0,3.45921,199.908461,904.85425
temperature,112815.0,14.086305,10.127431,-22.3,6.4,15.60000,22.400000,36.30000
humidity,112815.0,71.364783,19.339558,8.0,58.0,74.00000,87.000000,100.00000
dew_point,112815.0,8.422604,11.042804,-32.8,0.3,9.10000,18.100000,28.70000
wind_dir,112815.0,164.358392,114.488457,0.0,68.0,158.00000,270.000000,338.00000
wind_spd,112815.0,7.529428,6.144344,0.0,3.2,6.20000,10.500000,116.10000
uv_idx,112815.0,1.238754,2.142408,0.0,0.0,0.00000,2.000000,12.00000
visibility,112815.0,14.469508,5.984267,0.0,11.3,16.10000,16.100000,64.40000
cloudiness,112815.0,55.864495,37.811843,0.0,15.0,63.00000,92.000000,100.00000


In [5]:
# 이상치 탐색(IQR 방법 이용)
def get_outlier(df= None, columns = None, weight = 1.5):
    fraud = df[columns]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    iqr = quantile_75-quantile_25
    iqr_weight = iqr*weight
    lowest_val = quantile_25-iqr_weight
    highest_val = quantile_75+iqr_weight
    outlier_index = fraud[(fraud<lowest_val)|(fraud>highest_val)].index
    return outlier_index

In [6]:
cols = df.columns[3:]

for col in cols:
    outlier_index = get_outlier(df=df, columns = col, weight = 1.5)
    print(col+'의 이상치 데이터 인덱스 : ', outlier_index)

temperature의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
humidity의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
dew_point의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
wind_dir의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
wind_spd의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
uv_idx의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
visibility의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
cloudiness의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
ceiling의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
pressure의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')
precip_1h의 이상치 데이터 인덱스 :  Int64Index([], dtype='int64')


## 2. 결측치 처리

In [7]:
# 결측치 확인
df.isna().sum()

id                  0
time                0
amount              0
temperature    122194
humidity       122194
dew_point      122194
wind_dir       122194
wind_spd       122194
uv_idx         122194
visibility     122194
cloudiness     122194
ceiling        122194
pressure       122199
precip_1h      122194
dtype: int64

In [8]:
# 선형 보간법으로 결측치 처리
tmp_df1 = df.iloc[:,:3]
tmp_df2 = df.iloc[:,3:].interpolate()
df = pd.concat([tmp_df1, tmp_df2], axis=1)

In [9]:
df.isna().sum()

id             0
time           0
amount         0
temperature    4
humidity       4
dew_point      4
wind_dir       4
wind_spd       4
uv_idx         4
visibility     4
cloudiness     4
ceiling        4
pressure       4
precip_1h      4
dtype: int64

In [10]:
# 남은 결측치에 대해서 뒤쪽 값으로 채우기
df = df.fillna(method='bfill')

In [11]:
# 소수점 정리
for col in df.columns[2:]:
    if(col == 'amount'):
        df[col] = df[col].round(3)
    else:
        df[col] = df[col].round(1)

# 정렬
del df['pressure']
df = df.sort_values(by=['id', 'time'])
df

,id,time,amount,temperature,humidity,dew_point,wind_dir,wind_spd,uv_idx,visibility,cloudiness,ceiling,precip_1h
0,1,2020-06-01 01:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
1,1,2020-06-01 02:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
2,1,2020-06-01 03:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
3,1,2020-06-01 04:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
4,1,2020-06-01 05:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235004,24,2021-07-23 20:00:00+09:00,7.819,26.5,82.0,23.3,270.0,1.4,0.0,16.1,28.0,11278.0,0.0
235005,24,2021-07-23 21:00:00+09:00,0.067,26.5,82.0,23.3,270.0,1.4,0.0,16.1,28.0,11278.0,0.0
235006,24,2021-07-23 22:00:00+09:00,0.101,26.5,82.0,23.3,270.0,1.4,0.0,16.1,28.0,11278.0,0.0
235007,24,2021-07-23 23:00:00+09:00,0.134,26.5,82.0,23.3,270.0,1.4,0.0,16.1,28.0,11278.0,0.0


In [12]:
df_11 = df[df['id'] == 11]
df_11['id'] = 24
df = df.drop(df[df['id'] == 24].index[0:])
df = pd.concat([df, df_11])

C:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
df

,id,time,amount,temperature,humidity,dew_point,wind_dir,wind_spd,uv_idx,visibility,cloudiness,ceiling,precip_1h
0,1,2020-06-01 01:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
1,1,2020-06-01 02:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
2,1,2020-06-01 03:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
3,1,2020-06-01 04:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
4,1,2020-06-01 05:00:00+09:00,0.000,16.6,97.0,16.2,270.0,11.7,0.0,4.8,91.0,884.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227828,24,2021-07-23 20:00:00+09:00,7.581,30.2,64.0,22.7,45.0,2.2,0.0,16.1,41.0,6096.0,0.0
227829,24,2021-07-23 21:00:00+09:00,0.000,29.5,65.0,22.3,90.0,4.3,0.0,16.1,79.0,9144.0,0.0
227830,24,2021-07-23 22:00:00+09:00,0.000,28.8,62.0,20.9,0.0,3.6,0.0,20.9,76.0,9144.0,0.0
227831,24,2021-07-23 23:00:00+09:00,0.000,28.2,72.0,22.9,338.0,3.6,0.0,16.1,92.0,9144.0,0.0


In [14]:
df.to_csv(path + "Preprocessing_data.csv", index = False)